# How to learn feature for functional maps

In this notebook, we show how to use deep functional maps to learn feature for 3d shape matching.

In [1]:
import os

os.environ["GEOMSTATS_BACKEND"] = "pytorch"

import torch
from torch.utils.data import DataLoader, random_split

from geomfum.dataset.torch import PairsDataset, ShapeDataset
from geomfum.descriptor.learned import FeatureExtractor
from geomfum.forward_functional_map import ForwardFunctionalMap
from geomfum.learning.losses import (
    BijectivityLoss,
    GeodesicError,
    LaplacianCommutativityLoss,
    LossManager,
    OrthonormalityLoss,
)
from geomfum.learning.models import FMNet
from geomfum.learning.trainer import DeepFunctionalMapTrainer


First, we define our model. We can instantiate it combining feature extractors and forward logic, however, we provide some classic frameworks, like FMNet.

In [2]:
# Build the model
fmap_module = ForwardFunctionalMap(1e3, 1, True)

feature_extractor = FeatureExtractor.from_registry(
    which="diffusionnet",
    cache_dir="../../../datasets/faust/train_set/diffusion",
    device="cuda",
)

fumctional_map_model = FMNet(
    feature_extractor=feature_extractor, fmap_module=fmap_module
)


Then, we instantiate the training dataset and we split it for training purposes.

In [ ]:
# build the train and test loaders
TRAIN_SET_PATH = "../../../datasets/faust/train_set/"
dataset = ShapeDataset(
    TRAIN_SET_PATH, spectral=True, distances=True, device="cuda", k=30
)
train_size = int(0.8 * len(dataset))  # we split the shapes into train and validation
val_size = len(dataset) - train_size

train_shapes, validation_shapes = random_split(dataset, [train_size, val_size])
# we create a dataset of pairs from the training shapes
train_dataset = PairsDataset(
    train_shapes,
    pair_mode="all",
    n_pairs=100,
)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# we create a dataset of pairs from the test shapes
validation_dataset = PairsDataset(
    validation_shapes,
    pair_mode="all",
    n_pairs=100,
)
val_loader = DataLoader(validation_dataset, batch_size=1, shuffle=True)

C:\Users\giuli\OneDrive\Research\geomfum_proj\geomfum\geomfum\_backend\pytorch\sparse.py:22: UserWarning: Sparse CSC tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  return _torch.sparse_csc_tensor(ccol_indices, row_indices, values, size=array.shape)


we build optimizer

In [ ]:
optimizer = torch.optim.Adam(fumctional_map_model.parameters(), lr=1e-3)


Now we define the losses that we will cnsider. Again we can define our own losses, howver we provide some classic functional map energies, like the orthonormality loss.

In [ ]:
# define the loss
losses = [
    OrthonormalityLoss(weight=1.0),
    BijectivityLoss(weight=1.0),
    LaplacianCommutativityLoss(weight=1e-3),
]
loss_manager = LossManager(losses)

losses = [
    GeodesicError(),
]

val_loss_manager = LossManager(losses)

We have defined a trainer for simplicity that thakes as input model, losses, train and val datasets and optimizer and manages the training loops.

In [ ]:
trainer = DeepFunctionalMapTrainer(
    model=fumctional_map_model,
    train_loss_manager=loss_manager,
    val_loss_manager=val_loss_manager,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
)

In [ ]:
trainer.train()

INFO: Epoch [1/100] - Training
Epoch 1/100 (Train):   0%|          | 0/4032 [00:38<?, ?batch/s]


KeyError: 'D'

In [ ]:
# to access the trained feature extractor, you can use:
trained_model = trainer.model.feature_extractor
# and we can save it as
trained_model.save("ULSSM_faust.pth")

In [ ]:
import scipy

A = scipy.io.loadmat("../../../datasets/faust/train_set/dist/tr_reg_026.mat")

In [ ]:
import scipy.io


scipy.io.savemat("../../../datasets/faust/train_set/dist/tr_reg_026.mat", A[]

In [ ]:
A["D"]

KeyError: 'D'